<a href="https://colab.research.google.com/github/AbishekArunachalam/Image-Captioning--Deep-Learning-model-/blob/master/DeepLearningAT3_Building_an_Image_Captioning_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning - Image labelling model

### Install packages

In [1]:
# Install the packages
!pip install -U -q PyDrive
!pip install keras

     |████████████████████████████████| 993kB 6.7MB/s 


### Import packages

In [2]:
# Import packages
import os
import keras
import numpy as np
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from keras.applications.resnet50 import preprocess_input, decode_predictions

Using TensorFlow backend.


### Mount data

In [3]:
# Mount data from Google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
google_auth = GoogleAuth()
google_auth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(google_auth)

In [0]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

Load VGG model with pretrained weights.

In [6]:
# load the model
model = VGG16()

# summarise model
model.summary()

W0616 14:57:52.738249 139699512551296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0616 14:57:52.792993 139699512551296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0616 14:57:52.801836 139699512551296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0616 14:57:52.849467 139699512551296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



553467904/553467096 [==============================] - 21s 0us/step


W0616 14:58:14.798727 139699512551296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0616 14:58:14.800159 139699512551296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Remove the last layer from the VGG model.

In [7]:
# remove the prediction layer from the model
model.layers.pop()

# summarise model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

There are no non-trainable parameters as we have removed the prediction layer.

In [8]:
# instantiate model with input and output tensor
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)

# summarize model
print(model.summary()) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
'''
extracts features from each photo in the directory

Input:
path of the directory

Returns:
dictionary of file name as key and feature as value

'''
def extract_features(directory):
  
  # instantiate a dictionary object to store the features
  features = dict()
  
  for name in listdir(directory):
    
    # load an image from file
    filename = directory + '/' + name
    
    # resize to 224*224 image to train on VGG model
    image = load_img(filename, target_size=(224, 224))
    
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    
    # reshape array to take images in batches
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    
    # normalise array based on the model
    image = preprocess_input(image)
    
    # extract features
    feature = model.predict(image, verbose=0)
    
    # get the names of the image file
    image_id = name.split('.')[0]
    
    # pass file name as key and extracted features as value
    features[image_id] = feature
    
  return features

In [11]:
# extract features from all images
directory = 'gdrive/My Drive/data/flickr8k_Dataset'
features = extract_features(directory)

print('Extracted Features: %d' % len(features))

# save to file
dump(features, open('features.pkl', 'wb'))

Extracted Features: 8110


### Prepare text data

In [0]:
import string

'''
load doc into memory

Input:
filename

Returns:
text in the file
'''
def load_doc(filename):
  
	# open the file as read only
	file = open(filename, 'r')
  
	# read all text
	text = file.read()
  
	# close the file
	file.close()
  
	return text

In [0]:
'''
# extract descriptions for images

Input: 
text in Flickr8k.token.txt file

Returns:
dictionary of image_id and image descriptions
'''

def load_descriptions(doc):
  
  mapping = dict()
  
  # loop through each line
  for line in doc.split('\n'):
    
    # split line by white space
    tokens = line.split()
    
    # skip lines that has length <2
    if len(line) < 2:
      continue
    
    # take the first token as the image id, the rest as the description
    image_id, image_desc = tokens[0], tokens[1:]
    
    # extract filename from image_id
    image_id = image_id.split('.')[0]
    
    # convert description tokens back to string with white spaces
    image_desc = ' '.join(image_desc)
    
    # create a list for each image_id
    if image_id not in mapping:
      mapping[image_id] = list()
      
    # store description
    mapping[image_id].append(image_desc)
    
  return mapping

In [0]:
'''
# clean descriptions

Input:
dictionary of image_id and image descriptions

'''

def clean_descriptions(descriptions):
  
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
  
  # loop through the dictionary
	for key, desc_list in descriptions.items():
    
    # loop through each item in the list
		for i in range(len(desc_list)):
      
			desc = desc_list[i]
      
			# tokenize
			desc = desc.split()
      
			# convert to lower case
			desc = [word.lower() for word in desc]
      
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
      
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
      
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
      
			# store as string
			desc_list[i] =  ' '.join(desc)

In [0]:
'''
# convert the loaded descriptions into a vocabulary of words

Input:
dictionary of image_id and image descriptions

Output:
list of words in the descriptions

'''

def to_vocabulory(descriptions):
  
  # build a list of a description strings
  all_desc = set()
  
  # loop through each key in desc dictionary
  for key in descriptions.keys():
    
    # add words in the description to list
    [all_desc.update(d.split()) for d in descriptions[key]]
    
  return all_desc

In [0]:
'''
# save descriptions to file, one per line

Input:
descriptions - dictionary of image_id and image descriptions
filename - name of the file to save descriptions

Output:

'''
def save_descriptions(descriptions, filename):
  
  # instantiate an empty list
	lines = list()
  
  # loop through description dictionary
	for key, desc_list in descriptions.items():
    
    # loop through description list
		for desc in desc_list:
      
      # concatenate image_id and cleaned descriptions 
      # in a single line and add it to list
			lines.append(key + ' ' + desc)
  
  # create new line for each list item
	data = '\n'.join(lines)
  
  # create file with write permission
	file = open(filename, 'w')
  
  # write data to file
	file.write(data)
  
  # close file
	file.close()

In [45]:
filename = 'gdrive/My Drive/data/flickr8k_text/Flickr8k.token.txt'

# load descriptions
doc = load_doc(filename)

# parse descriptions
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

# clean descriptions
clean_descriptions(descriptions)

# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

# save to file
save_descriptions(descriptions, 'descriptions.txt')

Loaded: 8092 
{'staff', 'surfers', 'africanamerican', 'away', 'duel', 'held', 'listening', 'nike', 'limbs', 'weather', 'exercises', 'disks', 'legos', 'frog', 'saver', 'flanked', 'bordered', 'painting', 'winces', 'blowing', 'sunning', 'cart', 'helps', 'tugofwar', 'flannel', 'series', 'celebrates', 'what', 'gyro', 'symbol', 'ponytailed', 'somone', 'hikes', 'neck', 'presenting', 'found', 'instruments', 'setup', 'top', 'heavily', 'visitor', 'dirty', 'umpire', 'chris', 'goods', 'squares', 'flakes', 'tinsel', 'steady', 'ringing', 'one', 'winter', 'lakefront', 'spiderman', 'best', 'indoor', 'pom', 'fedora', 'sheltered', 'attentive', 'pocket', 'shadowy', 'golf', 'whitebearded', 'platform', 'armoire', 'dappled', 'corner', 'closeup', 'underpants', 'hammer', 'examining', 'branded', 'bead', 'creative', 'pinstripe', 'blackclad', 'bull', 'railgrind', 'stoop', 'vegas', 'decorative', 'distorted', 'overcast', 'backsides', 'punkish', 'sparking', 'move', 'scrimmage', 'breakdancer', 'bundled', 'greysuited

# Develop Deep Learning Model

### Loading data

In [0]:
from pickle import load
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [0]:
# load a pre-defined list of photo identifiers
def load_set(filename):
  
	doc = load_doc(filename)
  
	dataset = list()
  
	# process line by line
	for line in doc.split('\n'):
    
		# skip empty lines
		if len(line) < 1:
			continue
      
		# get the image identifier
		identifier = line.split('.')[0]
    
		dataset.append(identifier)
    
	return set(dataset)

In [0]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

In [0]:
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

In [54]:
# load training dataset (6K)
filename = 'gdrive/My Drive/data/flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('features.pkl', train)
print('Photos: train=%d' % len(train_features))

Dataset: 6000
Descriptions: train=6000
Photos: train=6000


In [59]:
pip install nltk

In [58]:
from tokenize import tokenize

# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

NameError: ignored